<a href="https://colab.research.google.com/github/Hurabashi/Matricu_Daugyba_su_Strassen/blob/main/Matricu_daugyba_vykdymo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Šiame projekte lyginsime paprastą matricų sandaugos algoritmą ir matricų sandaugos Strassen algoritmą.

Pirma pradėsime nuo paprastos matricų sandaugos.

In [ ]:
def matricu_sandauga_paprasta(matrica_1,matrica_2):
  '''
  Iprastu ir neoptimizuotu budu sudauginame dvi matricas, veikia jeigu sutampa
  pirmosios plotis ir antrosios aukstis/ilgis.
  '''
  suma = 0
  galutine_matrica = []
  for eilute in matrica_1:
    ilgis_eil = len(eilute)
    ilgis_stulp = len(matrica_2[0])
    galutines_matricos_eilute = []
    for i in range(ilgis_stulp):
      for j in range(ilgis_eil):
        suma = suma + eilute[j] * matrica_2[j][i]
      galutines_matricos_eilute.append(suma)
      suma = 0 
    galutine_matrica.append(galutines_matricos_eilute)
  return galutine_matrica
  
  

Paruosiame matricas tinkamas Strassen algoritmui, jeigu jos nebūtų kvadratinės ir/arba 2^n. Mes pridedame nulių, kad jas tokiomis paversti.
(Pastaba: užduoty naudojamos tik kvadratinės matricos, čia veiks su visomis daugintinomis matricomis)

In [ ]:
def matricu_kvadratinimas(matrica_1, matrica_2):
  '''
  Is abieju matricu randame didziausia dimensija, tada randame tai dimensijai
  maziausia 2^n, ir tada abieju matricu dimensijas ispleciame iki jos, pridedant
  nulius.
  '''
  #ieskome kokia didziausia dimensija
  didziausia_dimensija = max(len(matrica_1[0]),len(matrica_1),
                    len(matrica_2[0]),len(matrica_2))
  #padarome didziausia dimensija i 2^n 
  while (didziausia_dimensija & (didziausia_dimensija - 1)) != 0:
         didziausia_dimensija = didziausia_dimensija + 1

  #paplatiname matricas iki didziausios dimensijos
  #prailginame eilutes nuliais
  while len(matrica_1[0]) != didziausia_dimensija:
    for eilute in matrica_1:
      eilute.append(0)
  while len(matrica_2[0]) != didziausia_dimensija:
    for eilute in matrica_2:
      eilute.append(0)
  
  #sukuriam nuliu eilutes
  nuliai = []
  for i in range(didziausia_dimensija):
    nuliai.append(0)
  
  #prideda nuliu eilutes
  while len(matrica_1) != didziausia_dimensija:
    matrica_1.append(nuliai)
  while len(matrica_2) != didziausia_dimensija:
    matrica_2.append(nuliai)


 

Toliau mes apsibrėžiame pagalbines funkcijas: matricų sudėtį, atimtį, išardymą į 4 mažesnias matricas, bei sujungimo atgal į vieną. Aišku tai reikalinga, nes mes nesinaudojame išorinėmis bibliotekomis kaip numpy, kurios jau turi apibrėžtą dalį operacijų bei palengvina kitas, bet su jomis algoritmų sudėtingumo lyginimas taptų gan beprasmis, nes jos parašytos kitomis programavimo kalbomis ir tai nebūtų savarankiškas darbas.

In [ ]:
def sudetis(x, y):
  #tikrinama ar musu atveju tai matrica jei taip atlieka matricu sudetis
  if isinstance(x, list):
    c = []
    plotis = len(x[0])
    ilgis = len(x)
    for i in range(ilgis):
      c_eil = []
      for j in range(0, plotis): 
        c_eil.append(x[i][j] + y[i][j])
      c.append(c_eil)
    return c
  else:
    return x + y

def atimtis(x, y):
  #tikrinama ar musu atveju tai matrica jei taip tai atliekama matricu atimtis
  if isinstance(x, list):
    c = []
    plotis = len(x[0])
    ilgis = len(x)
    for i in range(ilgis):
      c_eil = []
      for j in range(0, plotis): 
        c_eil.append(x[i][j] - y[i][j])
      c.append(c_eil)
    return c
  else:
    return x - y

def matricos_padalinimas(matrica):
  #padalinime matrica i 4 matricas
  dimensija = len(matrica)
  dimensija_galutine = int(dimensija/2)

  mat_00 = []
  for eil in matrica[0 : dimensija_galutine]:
    mat_00.append(eil[0 : dimensija_galutine])
    
  mat_01 = []
  for eil in matrica[0 : dimensija_galutine]:
    mat_01.append(eil[dimensija_galutine : dimensija])
    
  mat_10 = []
  for eil in matrica[dimensija_galutine : dimensija]:
    mat_10.append(eil[0 : dimensija_galutine])
    
  mat_11 = []
  for eil in matrica[dimensija_galutine : dimensija]:
    mat_11.append(eil[dimensija_galutine : dimensija])
  return mat_00, mat_01, mat_10, mat_11
  
def matricos_suvienijimas(mat_00, mat_01, mat_10, mat_11):
  #sujungiame 4 matricas i viena
  if isinstance(mat_00, list):
    c = []
    dimensija = len(mat_00)
    for i in range(dimensija):
      c.append(mat_00[i] + mat_01[i])   
    for i in range(dimensija):
      c.append(mat_10[i] + mat_11[i])
  else:
    c = [[mat_00, mat_01],
        [mat_10, mat_11]]
    
  return c

Pagaliau galime apsibrėžti Strassen rekursinį matricų daugybos algoritmą.

In [ ]:
def matricu_sandauga_strassen(x, y):
  '''
  Dauginame matricas Strassen rekurisniu algoritmu
  '''

  #sukuriame isejimo salyga pasiekus skaliarus
  if len(x) == 1:
    return x[0][0] * y[0][0]

  #padaliname matricas i savus kvadratus
  x_00, x_01, x_10, x_11 = matricos_padalinimas(x)
  y_00, y_01, y_10, y_11 = matricos_padalinimas(y)

  #sukuriame konstantas reikalingas Strassen algoritmui
  m_1 = matricu_sandauga_strassen(atimtis(x_01, x_11), sudetis(y_10, y_11))
  m_2 = matricu_sandauga_strassen(sudetis(x_00, x_11), sudetis(y_00, y_11))
  m_3 = matricu_sandauga_strassen(atimtis(x_00, x_10), sudetis(y_00, y_01))
  m_4 = matricu_sandauga_strassen(sudetis(x_00, x_01), y_11)
  m_5 = matricu_sandauga_strassen(x_00, atimtis(y_01, y_11))
  m_6 = matricu_sandauga_strassen(x_11, atimtis(y_10, y_00))
  m_7 = matricu_sandauga_strassen(sudetis(x_10, x_11), y_00)

  c_00 = sudetis(sudetis(m_1, m_2), atimtis(m_6, m_4))
  c_01 = sudetis(m_4, m_5)
  c_10 = sudetis(m_6, m_7)
  c_11 = sudetis(atimtis(m_2, m_3), atimtis(m_5, m_7))

  #sujungiame i didesne matrica
  c = matricos_suvienijimas(c_00, c_01, c_10, c_11)
  return c

Dabar sukurkime galimybę nuskaityti matricas iš failų ir atlikti daugybą. 

In [ ]:
failas = 'matricos.txt'
with open(failas, "r") as failas:
    FileasList = failas.readlines()

matrica_did = []

for i in range(len(FileasList)):
  eilute = list(map(int, FileasList[i].split()))
  matrica_did.append(eilute)
  if eilute == []:
    kitos_matricos_zyma = i
  

matrica_1 = matrica_did[0:kitos_matricos_zyma]
matrica_2 = matrica_did[kitos_matricos_zyma+1:]

print('Pasirinkite kokia daugyba norite atlikti Strassen ar paprasta su nuskaitytomis matricomis is failo matricos.txt')
print('Jeigu Strassen, rasykite s simboli, jeigu paprasta - p simboli arba bet ka kita:')
simbolis = input()

if(simbolis == 's'):
  matricu_kvadratinimas(matrica_1,matrica_2)
  galutine_matrica = matricu_sandauga_strassen(matrica_1, matrica_2)
else:
  galutine_matrica = matricu_sandauga_paprasta(matrica_1, matrica_2)

print('Atsakymas:')
for eilute in galutine_matrica:
  print(eilute)
input()